# Equilibrium calculations with MELTS and MAGEMin

The following notebook demonstrates how we performed equilibrium calculations using experimental compositions and conditions (P, T, X). Initial calculations were performed using the pMELTS, rhyolite-MELTS v1.0.2, and rhyolite-MELTS v1.2.0 thermodynamic models. These calculations are performed in Python using the PetThermoTools (v0.2.31) and alphaMELTS for Python (v2.3.1) packages. Detailed instructions for the installation of PetThermoTools and alphaMELTS for Python can be found on the PetThermoTools ReadtheDocs page (https://petthermotools.readthedocs.io/en/latest/Installation/InstallationScript.html). 

To assess the performance of other thermodynamic models calculations were also performed using the Holland et al. (2018), Green et al. (submitted), and Weller et al. (2024) thermodynamic models. This is achieved through the Julia packages MAGEMin_C (https://github.com/ComputationalThermodynamics/MAGEMin_C.jl) and the 'connector' package MAGEMinCalc (v0.4.0) which is designed to facilitate MAGEMin calculations from Python. For calculations performed using the Green et al. and Weller et al. thermodynamoc models we use MAGEMin_C v1.6.9, with an older version (v1.4.9 in this case) required to access the original Holland et al. 2018 thermodynamic model. As such, only calculations for the Green et al. and Weller et al. models are shown here.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PetThermoTools as ptt
import pickle
import os
import sys

# this can be useful for visualizing the phase diagrams - hover over the plots to see the phase assemblage
%matplotlib widget

# this allows figures to be saved as svg files with the text preserved
plt.rcParams['svg.fonttype'] = 'none'

# check the version
ptt.__version__


'0.2.31'

Calculations performed using the alphaMELTS for Python package can produce a huge number of outputs that are all printed in the notebook if you are running a Mac. Run the following code twice to suppress these outputs.

In [3]:
#### Suppress extensive MELTS outputs, RUN TWICE ####
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

If you want to perform calculations using the Green, Holland, or Weller thermodynamic models it is good to first check the pyjulia installation and compile the installed julia modules:

In [4]:
import julia
julia.install()
from julia.api import Julia
jl = Julia(compiled_modules=False)

## Load experimental data
This has already been filtered to include only values with known fO2.

In [5]:
Data = pd.read_excel('df_for_MELTS_H2O_fo2.xlsx')
Data = Data.fillna(0.0)


## Use Kress and Carmichael to convert fO2 into Fe3Fet ratios 

This will be used as input to the calculations rather than raw fO2 values or offsets from a buffer 

In [6]:
import Thermobar as pt
Fe3Fet_Liq=pt.convert_fo2_to_fe_partition(liq_comps = Data.loc[:,['SiO2_Liq', 'TiO2_Liq', 'Al2O3_Liq', 'FeOt_Liq',
                                                        'Cr2O3_Liq', 'MnO_Liq', 'MgO_Liq', 'CaO_Liq', 
                                                        'Na2O_Liq', 'K2O_Liq', 'P2O5_Liq', 'H2O_Liq']], T_K = Data['T_C']+273.15,
                               P_kbar = Data['P_GPa']*10, fo2 = 10**Data['logfO2'],
                               renorm=False, model = "Kress1991")['Fe3Fet_Liq']

## Equilibrate calculations for each model

In this cell block the code will produce an excel sheet for each thermodynamic model containing the calculation results.

In [8]:
model = ["MELTSv1.2.0", "MELTSv1.0.2", "pMELTS", "Green2025", "Weller2024"]
for m in model:
    Combined = ptt.equilibrate_multi(Model = m,
                                    bulk = Data,
                                    T_C = Data['T_C'].astype(float).values,
                                    P_bar = Data['P_GPa'].astype(float).values*10000.0,
                                    Fe3Fet_Liq= Fe3Fet_Liq.astype(float).values,
                                    copy_columns=["Experiment", "Citation"])
    

    with pd.ExcelWriter(m + '_filt_withfO2_H2O.xlsx', engine='xlsxwriter') as writer:
        Combined.to_excel(writer, sheet_name='All Data', index=False)

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]